In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import scikitplot as skplt
import numpy as np
import xgboost as xgb
import seaborn as sns
import joblib
%matplotlib inline

from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.calibration import calibration_curve
# from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_precision_recall_curve
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN
import lightgbm as lgb
from lightgbm import LGBMClassifier


# from sksurv.preprocessing import OneHotEncoder
# from sksurv.ensemble import RandomSurvivalForest

import warnings  
warnings.filterwarnings("ignore")  

from imblearn.over_sampling import SMOTE
from collections import Counter

D:\Users\122156\Anaconda3\envs\tensorflow\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_ID_score = pd.read_excel("../CXR/train_test_split.xlsx", sheet_name = 'train')
test_ID_score = pd.read_excel("../CXR/train_test_split.xlsx", sheet_name = 'test')

train_ID_list = [int(i.split('_')[0]) for i in list(train_ID_score.ID)]
test_ID_list = [int(i.split('_')[0]) for i in list(test_ID_score.ID)]

In [3]:
dataset = pd.read_csv('../concat_influ8_severity_score.csv')
dataset = dataset[['ID', 'Gender', 'Age', 'BMI', 'Diastolic_blood_pressure', 'Heart_rate', 
                   'Respiratory_rate', 'Creatinine', 'CRP', 'Hemoglobin', 'Bicarbonate', 
                   'PCO2', 'pH', 'Platelet', 'PO2',  'Blood_urea_nitrogen', 
                   'Lactic_acid', 'INR', 'Glucose', 'Hematocrit', 'severity_score', 'Survival_30']]
train = dataset.loc[dataset['ID'].isin(train_ID_list)]  #534筆(佔69.8%)
train = train.drop('ID', axis = 1)
test = dataset.loc[dataset['ID'].isin(test_ID_list)]  #231筆(佔30.2%)
test = test.drop('ID', axis = 1)

In [4]:
external_dataset = pd.read_csv("../concat_italy_severity_score.csv")
external_dataset = external_dataset[['Gender', 'Age', 'BMI', 'DBP (mmHg)', 'Heart rate ', 'RR', 
                                  'Creatinine ', 'CRP', 'Hb', 'HCO3-', 'pCo2', 'Ph',
                                  'PLT', 'pO2', 'BUN ', 'Lactate', 'INR ', 'Glycemia', 
                                  'HCT', 'severity_score','30 days mortality (0 alive, 1 death)']]
external_dataset['Gender'] = external_dataset['Gender'].replace('F',0).replace('M',1)
external_dataset['BMI'] = external_dataset['BMI'].replace("Normal", "18.5")
external_dataset['BMI'] = external_dataset['BMI'].replace("Overweight", "21.75")
external_dataset['BMI'] = external_dataset['BMI'].replace("Obese ", "27.5")
external_dataset['BMI'] = external_dataset['BMI'].replace("Underweight", "30")
external_dataset['Gender'] = pd.to_numeric(external_dataset['Gender'], errors='coerce')
external_dataset['BMI'] = pd.to_numeric(external_dataset['BMI'], errors='coerce')

In [ ]:
external_dataset = external_dataset.fillna(round(external_dataset.mean(), 2))

## Random Forest

In [ ]:
from tqdm import tqdm
import pickle
best_auc = best_sensitivity = best_precision = 0
best_diff = 0.0

### Resampling
count_class_0, count_class_1 = train.Survival_30.value_counts()
df_class_0 = train[train['Survival_30'] == 0]
df_class_1 = train[train['Survival_30'] == 1]
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

### validation ###
X_train_pred = df_test_over.drop('Survival_30', axis = 1)
y_train_pred = df_test_over[['Survival_30']]
X_test_pred = test.drop('Survival_30', axis = 1)
y_test_pred = test[['Survival_30']]

rfc = RandomForestClassifier(criterion = 'gini', max_features = 'auto', max_depth = 30, 
                 min_samples_leaf = 30, n_estimators = 70, random_state = 1,  oob_score = True)
rfc.fit(X_train_pred, y_train_pred)
y_prob_pred = rfc.predict_proba(X_test_pred)
y_pred_pred = rfc.predict(X_test_pred)

cm_pred = metrics.confusion_matrix([x for [x] in y_test_pred.values.tolist()], y_pred_pred)
tn_pred, fp_pred, fn_pred, tp_pred = cm_pred.ravel()
sensitivity_pred = tp_pred / (tp_pred+fn_pred+1e-8)
specificity_pred = tn_pred / (tn_pred+fp_pred+1e-8)
precision_pred = tp_pred/(tp_pred+fp_pred+1e-8)
recall_pred = tp_pred/(tp_pred+fn_pred+1e-8)
f1_pred = (2*precision_pred*recall_pred)/(precision_pred+recall_pred)
auc_pred = metrics.roc_auc_score(y_test_pred, y_prob_pred[:, 1])
acc_pred = metrics.accuracy_score(y_test_pred, y_pred_pred)

### external test ###
X_test_ex = external_dataset.drop('30 days mortality (0 alive, 1 death)', axis = 1)
y_test_ex = external_dataset[['30 days mortality (0 alive, 1 death)']]

y_prob_ex = rfc.predict_proba(X_test_ex)
y_pred_ex = rfc.predict(X_test_ex)

cm_ex = metrics.confusion_matrix([x for [x] in y_test_ex.values.tolist()], y_pred_ex)
tn_ex, fp_ex, fn_ex, tp_ex = cm_ex.ravel()
sensitivity_ex = tp_ex / (tp_ex+fn_ex+1e-8)
specificity_ex = tn_ex / (tn_ex+fp_ex+1e-8)
precision_ex = tp_pred/(tp_ex+fp_ex+1e-8)
recall_ex = tp_pred/(tp_ex+fn_ex+1e-8)
f1_ex = (2*precision_ex*recall_ex)/(precision_ex+recall_ex)
auc_ex = metrics.roc_auc_score(y_test_ex, y_prob_ex[:, 1])
acc_ex = metrics.accuracy_score(y_test_ex, y_pred_ex)
#################################

### validation ###
print("validation")
print("train_test_split_random_state:", 82, "model_random_state:", j)
print('AUC: {:.4f}'.format(auc_pred))
print('Acc: {:.4f}'.format(acc_pred))
print('sensitivity: {:.4f}'.format(sensitivity_pred))
print('specificity: {:.4f}'.format(specificity_pred))
print('precision: {:.4f}'.format(precision_pred))
print('recall: {:.4f}'.format(recall_pred))
print('F1: {:.4f}'.format(f1_pred))

fpr_pred, tpr_pred, thresholds_pred = metrics.roc_curve(y_test_pred, y_prob_pred[:,1])
AUC_PR_pred = metrics.auc(tpr_pred, fpr_pred)
print('The area under the recall precision curve: {:.4f}'.format(AUC_PR_pred))

print(classification_report(y_test_pred.values.tolist(), y_pred_pred, labels=[0,1]))

### Confusion Matrics
sns.set(font_scale = 1.4) # for label size
sns.heatmap(cm_pred, annot=True, annot_kws={"size": 16},fmt='g') # font size
plt.show()

### Recall Precision curves
disp_pred = plot_precision_recall_curve(rfc, X_test_pred, y_test_pred)
disp_pred.ax_.set_title('2-class Precision-Recall curve')
disp_pred.ax_.legend(loc = 'upper right')
plt.show()

### ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_pred, tpr_pred, 'b', label = 'AUC = %0.2f' % roc_auc_score(y_test_pred, y_prob_pred[:, 1]))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### external test ###
print("external test")
print("train_test_split_random_state:", 82, "model_random_state:", j)
print('AUC: {:.4f}'.format(auc_ex))
print('Acc: {:.4f}'.format(acc_ex))
print('sensitivity: {:.4f}'.format(sensitivity_ex))
print('specificity: {:.4f}'.format(specificity_ex))
print('precision: {:.4f}'.format(precision_ex))
print('recall: {:.4f}'.format(recall_ex))
print('F1: {:.4f}'.format(f1_ex))

fpr_ex, tpr_ex, thresholds_ex = metrics.roc_curve(y_test_ex, y_prob_ex[:,1])
AUC_PR_ex = metrics.auc(tpr_ex, fpr_ex)
print('The area under the recall precision curve: {:.4f}'.format(AUC_PR_ex))

print(classification_report(y_test_ex.values.tolist(), y_pred_ex, labels=[0,1]))

### Confusion Matrics
sns.set(font_scale = 1.4) # for label size
sns.heatmap(cm_ex, annot=True, annot_kws={"size": 16},fmt='g') # font size
plt.show()

### Recall Precision curves
disp_ex = plot_precision_recall_curve(rfc, X_test_ex, y_test_ex)
disp_ex.ax_.set_title('2-class Precision-Recall curve')
disp_ex.ax_.legend(loc = 'upper right')
plt.show()

### ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_ex, tpr_ex, 'b', label = 'AUC = %0.2f' % roc_auc_score(y_test_ex, y_prob_ex[:, 1]))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

model_file_name_pred='RF_AUC{:.4f}.pickle'.format(metrics.roc_auc_score(y_test_pred, y_prob_pred[:, 1]))
with open(model_file_name_pred, 'wb') as f:
    pickle.dump(rfc, f)